<a href="https://colab.research.google.com/github/Tycour/crisanti-toolshed/blob/main/docs/lessons/14_Introduction_to_Pandas_and_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Video: https://youtu.be/uR_uH7NKWjI

## Pandas

The primary data structures in pandas are implemented as two classes:

- `DataFrame`, which you can imagine as a relational data table, with rows and named columns.

- `Series`, which is a single column. 

A `DataFrame` contains one or more `Series` and a name for each `Series`.
The data frame is a commonly used abstraction for data manipulation. Similar implementations exist in Spark and R.

![alt text](https://storage.googleapis.com/lds-media/images/series-and-dataframe.width-1200.png)

Documentation: https://pandas.pydata.org/pandas-docs/stable/index.html

In [ ]:
import numpy as np
import pandas as pd


In [ ]:
data = {
    'apples': [3, 2, 0, 1], 
    'oranges': [0, 3, 7, 2],
    'pears': [1, 5, 4, 0]
}

In [ ]:
purchases = pd.DataFrame(data)
purchases

,apples,oranges,pears
0,3,0,1
1,2,3,5
2,0,7,4
3,1,2,0


In [ ]:
purchases = pd.DataFrame(data, index=['Mark', 'Robert', 'Lily', 'David'])
purchases

,apples,oranges,pears
Mark,3,0,1
Robert,2,3,5
Lily,0,7,4
David,1,2,0


In [ ]:
purchases.loc[(purchases['apples'] > 0) & (purchases['oranges'] > 0), ['apples', 'oranges']]

,apples,oranges
Robert,2,3
David,1,2


In [ ]:
purchases.iloc[1:,1:]

,oranges,pears
Robert,3,5
Lily,7,4
David,2,0


In [ ]:
purchases + 1

,apples,oranges,pears
Mark,4,1,2
Robert,3,4,6
Lily,1,8,5
David,2,3,1


In [ ]:
purchases.drop('David', axis=0)

# remove columns, rows
# subset

,apples,oranges,pears
Mark,3,0,1
Robert,2,3,5
Lily,0,7,4


In [ ]:
purchases = purchases + 0.5
purchases

,apples,oranges,pears
Mark,3.5,0.5,1.5
Robert,2.5,3.5,5.5
Lily,0.5,7.5,4.5
David,1.5,2.5,0.5


In [ ]:
purchases.apply(round, axis=0)

,apples,oranges,pears
Mark,4.0,0.0,2.0
Robert,2.0,4.0,6.0
Lily,0.0,8.0,4.0
David,2.0,2.0,0.0


In [ ]:
# read expression data from Taxiarchi et al. 2019
# premeiosis, start, meiosis, postmeiotic
# I, II, III, IV

# calculate mean expression in each population
# find genes that have peak expression in meiosis
# compare expression of testes-specific genes and ovaries-specific genes (Baker dataset)

# file in _data/ folder

df = pd.read_csv('expressions.csv')
df

## Merge

![alt text](https://data36.com/wp-content/uploads/2018/08/4-pandas-merge-inner-outer-left-right-1024x771.png)

In [ ]:
data_store_1 = {
    'apples': [3, 2, 0, 1], 
    'oranges': [0, 3, 7, 2],
    'pears': [1, 5, 4, 0]
}
purchases_store_1 = pd.DataFrame(data_store_1, index=['Mark', 'Robert', 'Lily', 'David'])

data_store_2 = {
    'milk': [3, 2, 0, 1, 5], 
    'chocolate': [0, 3, 7, 5, 9],
    'butter': [1, 5, 4, 3, 0],
    'prosecco': [0, 0, 0, 0, 10],
    'beer': [1, 2, 3, 1, 5],
}
purchases_store_2 = pd.DataFrame(data_store_2, index=['John', 'Robert', 'Lily', 'David', 'Marcus'])

In [ ]:
purchases_store_1

,apples,oranges,pears
Mark,3,0,1
Robert,2,3,5
Lily,0,7,4
David,1,2,0


In [ ]:
purchases_store_2

,milk,chocolate,butter,prosecco,beer
John,3,0,1,0,1
Robert,2,3,5,0,2
Lily,0,7,4,0,3
David,1,5,3,0,1
Marcus,5,9,0,10,5


In [ ]:
df = pd.merge(purchases_store_1, purchases_store_2, how='outer', left_on='name', right_on='customer')
df

,apples,oranges,pears,milk,chocolate,butter,prosecco,beer
David,1.0,2.0,0.0,1.0,5.0,3.0,0.0,1.0
John,NaN,NaN,NaN,3.0,0.0,1.0,0.0,1.0
Lily,0.0,7.0,4.0,0.0,7.0,4.0,0.0,3.0
Marcus,NaN,NaN,NaN,5.0,9.0,0.0,10.0,5.0
Mark,3.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN
Robert,2.0,3.0,5.0,2.0,3.0,5.0,0.0,2.0


In [ ]:
df = df.fillna(0)
df

,apples,oranges,pears,milk,chocolate,butter,prosecco,beer
David,1.0,2.0,0.0,1.0,5.0,3.0,0.0,1.0
John,0.0,0.0,0.0,3.0,0.0,1.0,0.0,1.0
Lily,0.0,7.0,4.0,0.0,7.0,4.0,0.0,3.0
Marcus,0.0,0.0,0.0,5.0,9.0,0.0,10.0,5.0
Mark,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
Robert,2.0,3.0,5.0,2.0,3.0,5.0,0.0,2.0


### Excercise
1. Find meiosis-specific genes - by looking at Tau value calculated from the expression data table

In [ ]:
# ------------ calculate tau

# read the file with expressions
df = pd.read_csv('data/FPKM_pops_means.hisat2_fc_deseq2.tsv', sep='\t')

# remove TXCHROM and TXSTRAND columns
df = df.drop(['TXCHROM', 'TXSTRAND'], axis=1)

# reshuffle the columns order
df = df.loc[:,['pre', 'start', 'meiosis', 'post', 'gene']]

# sent gene name as index and remove gene name column
df.index = df['gene']
df = df.drop('gene', axis=1)

# function to calculate tau value
# for the details refer to: https://academic.oup.com/bib/article/18/2/205/2562739
def calculate_tau(x):
    max_e = max(x)
    x_hats = []
    
    for tissue_e in x:
        if max_e > 0:
            x_hat = tissue_e / max_e
            x_hats.append(1 - x_hat)
        else:
            x_hats.append(1)
            
    tau = sum(x_hats) / (len(x) - 1)
    
    return tau
    
# create a copy of dataframe
df_annoted = df.copy()
# apply the function
df_annoted['tau'] = df_annoted.apply(calculate_tau, axis=1)
# define which cell population genes are specific for
# we set two conditions:
#  - tau > 0.7
#  - specificity is defined by having the highest expression
df_annoted['specific'] = df_annoted.loc[df_annoted['tau'] > 0.7,['pre', 'start', 'meiosis', 'post']].idxmax(axis=1)
df_annoted

###2. Find genes that have the peak of expression in meiosis

### 3. Find genes that are meiosis specific from the results of differential expression analysis 


> Differentially expressed genes are usually defined by being sigificantlly overexpressed or underexpressed when comparing them between two tissues. We can define the threshold by setting adjusted p value < 0.05 and logFC > 2.5 or logFC < -2.5. You can read more about how the significance is calculated and what log fold change is: https://www.nature.com/articles/ng1032z.pdf and https://genomebiology.biomedcentral.com/articles/10.1186/s13059-014-0550-8

You will find the results of differential expression in the folder `_data/rnaseq/CT_pops_diffexpr_*`

Each file has the following columns in addition to the column with a gene name:

----
```
baseMean log2FoldChange lfcSE stat pvalue padj
```

> The first column, `baseMean`, is a just the average of the normalized count values, dividing by size factors, taken over all samples. The remaining four columns refer to a specific contrast, namely the comparison of the levels DPN versus Control of the factor variable treatment. 

> The column `log2FoldChange` is the effect size estimate. It tells us how much the gene’s expression seems to have changed in comparison to control. This value is reported on a logarithmic scale to base 2: for example, a log2 fold change of 1.5 means that the gene’s expression is increased by a multiplicative factor of 2 1.5 ≈ 2.82. Of course, this estimate has an uncertainty associated with it, which is available in the column `lfcSE`, the standard error estimate for the log2 fold change estimate. 

> We can also express the uncertainty of a particular effect size estimate as the result of a statistical test. The purpose of a test for differential expression is to test whether the data provides sufficient evidence to conclude that this value is really different from zero. DESeq2 performs for each gene a hypothesis test to see whether evidence is sufficient to decide against the null hypothesis that there is no effect of the treatment on the gene and that the observed difference between treatment and control was merely caused by experimental variability (i. e., the type of variability that you can just as well expect between different samples in the same treatment group). As usual in statistics, the result of this test is reported as a p value, and it is found in the column `pvalue`. (Remember that a p value indicates the probability that a fold change as strong as the observed one, or even stronger, would be seen under the situation described by the null hypothesis.) We note that a subset of the p values in res are NA (“not available”). This is DESeq’s way of reporting that all counts for this gene were zero, and hence not test was applied. In addition, p values can be assigned NA if the gene was excluded from analysis because it contained an extreme count outlier.

> `padj` is p value adjusted for multiple testing

---
The way you can interpret which cell population is "treatment" and which "control" is that you always take the first cell population in the file name as the "treatment" and the second one as "control"

For example: `CT_pops_diffexpr_MEIOSIS_POST.hisat2_fc_deseq2.tab`
    - MEIOSIS is "treatment"
    - POST is "control"
So if you have a gene that has `log2FoldChange` value of `5` that means that there was 5-fold higher expression in `MEIOSIS` compared to `POST`.